In [8]:

import pandas as pd
import numpy as np
from service_aggregator import (
    get_embeddings_text,
    tokenizer,
    ft_model,
    embedder_text,
    get_clusters,
    get_topK_clusters,
    get_topK_news,
    get_filtered_by_role
)

/opt/homebrew/Caskroom/miniforge/base/envs/ml_env/lib/python3.8/site-packages/torch/serialization.py:602: UserWarning: 'torch.load' received a zip file that looks like a TorchScript archive dispatching to 'torch.jit.load' (call 'torch.jit.load' directly to silence this warning)
  warnings.warn("'torch.load' received a zip file that looks like a TorchScript archive"


In [24]:
# data = pd.read_csv("/Users/pavelkockin/Downloads/lenta-ru-news.csv")
data = pd.read_csv(
    "/Users/pavelkockin/Downloads/бухгалтер.csv", 
    sep=";",
    index_col=0,
)
data.head()

,uuid,url,title,post_dttm,processed_dttm,full_text,text_links
0,1ae58a3e-2f52-495f-9386-9f2c2b6abb49,https://www.consultant.ru/legalnews/20519,Разъяснили особенности учета занятости доброво...,2022-10-06 09:00:00,2022-10-07 20:25:07.958280,"Росстат указал, что мобилизованных и доброволь...",NaN
1,40a49284-62c2-448c-8c5e-6fe2b310f3ed,https://www.consultant.ru/legalnews/20515,В окончательном чтении приняли поправки об ипо...,2022-10-05 23:30:00,2022-10-07 20:25:07.958280,Предлагают размещать деньги в составе ипотечно...,NaN
2,84e95bdd-fc90-40ec-9c09-13804b469422,https://www.consultant.ru/legalnews/20507,Стали известны возможные коэффициенты-дефлятор...,2022-10-05 23:00:00,2022-10-07 20:25:07.958280,Минэкономразвития выставило на общественное об...,NaN
3,b9afb90a-d543-4efc-ae31-e68a96c42e40,https://www.consultant.ru/legalnews/20512,Подготовили поправки к перечням КБК на 2022 го...,2022-10-05 22:30:00,2022-10-07 20:25:07.958280,Проект проходит общественное обсуждение до 17 ...,NaN
4,d64af00b-f3b1-44f0-b73e-c22246b95740,https://www.consultant.ru/legalnews/20513,Планируют смягчить требования к энергоэффектив...,2022-10-05 21:30:00,2022-10-07 20:25:07.958280,Предложили при приобретении телевизоров с кодо...,NaN


In [25]:
data["post_dttm"] = pd.to_datetime(data["post_dttm"]) # + pd.DateOffset(hours  = 2)

In [32]:
corpus = data[(data["post_dttm"] >= "2022-09-01 00:00:00")]
print(corpus.shape)
corpus_embeddings = get_embeddings_text(
    corpus=corpus["full_text"],
    tokenizer=tokenizer,
    model_vectorizer=ft_model,
    model_embedder=embedder_text,
)
corpus["embedding"] = corpus_embeddings
print(np.array(corpus_embeddings).shape)

(656, 7)
(656, 128)


/var/folders/m0/9677s49d3bvfjvgs4x4b64w00000gn/T/ipykernel_16144/473585652.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  corpus["embedding"] = corpus_embeddings


In [33]:
buh_context = """Бухгалтер сегодня считается одной из самых достойных и нужных профессий. Бухгалтер – это специалист в области бухгалтерии, ведущий денежную и коммерческую отчетность на предприятиях.
Его задачи – это своевременная уплата налогов и сдача отчетности в государственные органы, отслеживание состояние счетов компании и правильное сведение баланса. 
Кадровый учет и делопроизводство, дополнительные соглашения к договорам также входят в обязанности бухгалтера. В небольших фирмах за это отвечает главный бухгалтер.
Бухгалтер отвечает за операционную поддержку бизнеса."""

In [34]:
filtered_corpus = get_filtered_by_role(
    corpus_embeddings=corpus_embeddings,
    role_embedding=get_embeddings_text([buh_context]),
)
corpus = corpus.iloc[filtered_corpus]

In [41]:
corpus["cluster"] = get_clusters(corpus["embedding"].to_list())

In [44]:
pd.set_option("max_colwidth", 120)
corpus[corpus["cluster"] == 0][["title","post_dttm","url"]]

,title,post_dttm,url
194,Применять мобильный телефон вместо ККТ нельзя,2022-09-15 12:00:00,https://www.consultant.ru/legalnews/20314
383,Правила применения ККТ при расчетах за маркированные товары могут упростить,2022-09-01 00:00:00,https://www.buhgalteria.ru/news/pravila-primeneniya-kkt-pri-raschetakh-za-markirovannye-tovary-mogut-uprostit.html


In [45]:
clusters_index = get_topK_clusters(corpus, K=3, date_col="post_dttm")

In [46]:
clusters_index

Int64Index([1, 2, 38], dtype='int64', name='cluster')

In [47]:
cluster_1 = corpus[corpus["cluster"] == 1].copy()

In [48]:
cluster_1[["post_dttm", "title", "url"]]

,post_dttm,title,url
120,2022-09-23 23:00:00,Обзор новых антикризисных мер за 17 – 23 сентября,https://www.consultant.ru/legalnews/20392
163,2022-09-19 15:05:00,Разработали проекты о контроле за соблюдением антикризисных мер и штрафах за их нарушение,https://www.consultant.ru/legalnews/20351
46,2022-09-19 00:00:00,Минфин подготовил законопроект о контроле за выполнением контрсанкционных требований,https://nalogoved.ru/news/12107.html
184,2022-09-20 00:00:00,У ФНС появятся новые «штрафные» полномочия,https://www.buhgalteria.ru/news/u-fns-poyavyatsya-novye-shtrafnye-polnomochiya.html


In [49]:
news_index = get_topK_news(cluster_1, K=2, date_col="post_dttm", embedding_col="embedding")

In [55]:
cluster_1.loc[news_index]

,uuid,url,title,post_dttm,processed_dttm,full_text,text_links,embedding,cluster,age,similarity,score
120,096b489f-cbb1-42ab-abc3-a224cf41ac0a,https://www.consultant.ru/legalnews/20392,Обзор новых антикризисных мер за 17 – 23 сентября,2022-09-23 23:00:00,2022-10-07 20:25:07.958280,Информацию обо всех важнейших решениях по стабилизации экономики и поддержке бизнеса см. в материале.Контроль за соб...,NaN,"[-0.033212121576070786, 0.048488542437553406, -0.035679157823324203, -0.023739363998174667, 0.016680771484971046, -0...",1,1.000000,0.874835,0.874835
184,43bd81f2-47e6-41a7-b9ab-a5e9d9ce8ccd,https://www.buhgalteria.ru/news/u-fns-poyavyatsya-novye-shtrafnye-polnomochiya.html,У ФНС появятся новые «штрафные» полномочия,2022-09-20 00:00:00,2022-10-08 02:51:41.076652,"Министерство финансов подготовило законопроекты, которые наделят Федеральную налоговую службу и Федеральную таможенн...","https://web.telegram.org/z/#-1612765597,https://vk.com/buhgalteriaru","[0.010317524895071983, 0.02046082727611065, -0.062315523624420166, 0.0075660329312086105, 0.008192973211407661, -0.0...",1,0.201681,0.899727,0.181458
